In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
384,tt1282140,8000000,74952305,Easy A,Emma Stone|Stanley Tucci|Amanda Bynes|Cam Giga...,Will Gluck,The rumour-filled totally false account of how...,After a little white lie about losing her virg...,92,Comedy,Screen Gems|Olive Bridge Entertainment,9/10/2010,6.6,2010
1639,tt0335345,30000000,611899420,The Passion of the Christ,Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...,Mel Gibson,"By his wounds, we were healed.","""The Passion of the Christ"" is a film about th...",127,Drama,Icon Productions,2/25/2004,6.8,2004
1086,tt0234215,150000000,738599701,The Matrix Reloaded,Keanu Reeves|Carrie-Anne Moss|Laurence Fishbur...,Lilly Wachowski|Lana Wachowski,Free your mind.,Six months after the events depicted in The Ma...,138,Adventure|Action|Thriller|Science Fiction,Village Roadshow Pictures|NPV Entertainment|He...,5/15/2003,6.6,2003
94,tt2473510,10000000,78099553,Paranormal Activity: The Ghost Dimension,Chris J. Murray|Brit Shaw|Ivy George|Lauren Fr...,Gregory Plotkin,You can't save them. All you can do is watch.,"Using a special camera that can see spirits, a...",88,Horror|Thriller,Paramount Pictures|Blumhouse Productions,10/21/2015,5.0,2015
1340,tt0388795,14000000,178043761,Brokeback Mountain,Heath Ledger|Jake Gyllenhaal|Randy Quaid|Miche...,Ang Lee,Love is a force of nature.,Brokeback Mountain is an Ang Lee film about tw...,134,Drama|Romance,River Road Entertainment|Alberta Film Entertai...,9/2/2005,7.2,2005


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [10]:
#creating a dictionary for answers
answers = {} 

#converting date to datetime instead of string
data['release_date'] = pd.to_datetime(data.release_date, format='%m/%d/%Y')

#making month a separated area
data['release_month'] = data.release_date.apply(lambda x: x.month)

#creating profit
data['profit'] = data.apply(lambda x: x.revenue - x.budget, axis=1)

#converting piped strings to set
def pipe_to_set(genres):
    if type(genres) != str:
        genres = ''
    genrelist = [str(x).strip() for x in genres.split('|')]
    return set(genrelist)

data['genre_set'] = (data.genres.apply(pipe_to_set))

data['director_set'] = (data.director.apply(pipe_to_set))

data['cast_set'] = (data.cast.apply(pipe_to_set))

data['company_set'] = (data.production_companies.apply(pipe_to_set))

data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

,budget,revenue,runtime,vote_average,release_year,release_month,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,6.933298,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,3.390196,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,1.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,4.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,7.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,10.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,12.000000,2.544506e+09


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [12]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' #+

In [11]:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,5,641683000,"{Fantasy, Adventure, Action}",{Rob Marshall},"{Geoffrey Rush, PenÃ©lope Cruz, Kevin McNally,...","{Moving Picture Company (MPC), Jerry Bruckheim..."


ВАРИАНТ 2

# 2. Какой из фильмов самый длительный (в минутах)?

In [15]:
answers['2'] = '2. Gods and Generals (tt0279111)' #+

In [14]:
data[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2003-02-21,5.8,2003,2,-43076064,"{War, Drama, History}",{Ronald F. Maxwell},"{C. Thomas Howell, Robert Duvall, Stephen Lang...","{Antietam Filmworks, Turner Pictures}"


# 3. Какой из фильмов самый короткий (в минутах)?





In [19]:
data[data.runtime == data.runtime.min()]


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,2011-04-13,6.8,2011,4,-15540000,"{Family, Animation}","{Don Hall, Stephen Anderson}","{Jim Cummings, Travis Oates, Craig Ferguson, B...","{Walt Disney Pictures, Walt Disney Animation S..."


In [20]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' #+

# 4. Какова средняя длительность фильмов?


In [21]:
data.runtime.mean()

109.6585494970884

In [22]:
answers['4'] = '2. 110' #+

# 5. Каково медианное значение длительности фильмов? 

In [23]:
data.runtime.median()

107.0

In [24]:
answers['5'] = '1. 107'#+

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [25]:
data[data.profit == data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,2009-12-10,7.1,2009,12,2544505847,"{Fantasy, Adventure, Action, Science Fiction}",{James Cameron},"{Stephen Lang, Zoe Saldana, Sam Worthington, S...","{Ingenious Film Partners, Lightstorm Entertain..."


In [92]:
answers['6'] = '5. Avatar (tt0499549)'#+

# 7. Какой фильм самый убыточный? 

In [28]:
data[data.profit == data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,7,-165710090,"{Adventure, Action, Western}",{Gore Verbinski},"{Helena Bonham Carter, James Badge Dale, Armie...","{Silver Bullet Productions (II), Jerry Bruckhe..."


In [93]:
answers['7'] = '5. The Lone Ranger (tt1210819)'#+

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [31]:
len(data[data.profit > 0])

1478

In [94]:
answers['8'] = '1. 1478'#+

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [33]:
data2 = data.query('release_year == 2008')
data2.query('profit == profit.max()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008,7,816921825,"{Crime, Drama, Thriller, Action}",{Christopher Nolan},"{Gary Oldman, Michael Caine, Heath Ledger, Chr...","{DC Comics, Warner Bros., Legendary Pictures, ..."


In [95]:
answers['9'] = '4. The Dark Knight (tt0468569)'#+

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [35]:
data2 = data.query('2012 <= release_year <= 2014')
data2.query('profit == profit.min()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,7,-165710090,"{Adventure, Action, Western}",{Gore Verbinski},"{Helena Bonham Carter, James Badge Dale, Armie...","{Silver Bullet Productions (II), Jerry Bruckhe..."


In [96]:
answers['10'] = '5. The Lone Ranger (tt1210819)'#+

# 11. Какого жанра фильмов больше всего?

In [37]:
data2 = pd.DataFrame([{y: 1 for y in x} for x in data.genres.str.split(r'\|')]).fillna(0)
data2.sum().sort_values(ascending=False)

Drama              782.0
Comedy             683.0
Thriller           596.0
Action             582.0
Adventure          415.0
Crime              315.0
Romance            308.0
Family             260.0
Science Fiction    248.0
Fantasy            222.0
Horror             176.0
Mystery            168.0
Animation          139.0
Music               64.0
History             62.0
War                 58.0
Western             19.0
Documentary          8.0
Foreign              2.0
dtype: float64

In [39]:
answers['11'] = '3. Drama'#+

ВАРИАНТ 2

In [38]:
data2 = pd.DataFrame(dict(data.genres.str.split(r'\|').apply(lambda x: Counter(x)))).fillna(0).transpose()
data2.sum().sort_values(ascending=False)

Drama              782.0
Comedy             683.0
Thriller           596.0
Action             582.0
Adventure          415.0
Crime              315.0
Romance            308.0
Family             260.0
Science Fiction    248.0
Fantasy            222.0
Horror             176.0
Mystery            168.0
Animation          139.0
Music               64.0
History             62.0
War                 58.0
Western             19.0
Documentary          8.0
Foreign              2.0
dtype: float64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [41]:
data[data.profit > 0].genre_set.apply(Counter).sum().most_common()

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

In [97]:
answers['12'] = '1. Drama'#+

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [44]:
data[['director_set', 'profit']].apply(
        lambda x: Counter({name: x.profit for name in x.director_set }), axis='columns'
    ).sum().most_common()


[('Peter Jackson', 5202593685),
 ('David Yates', 3379295625),
 ('Christopher Nolan', 3162548502),
 ('J.J. Abrams', 2839169916),
 ('Michael Bay', 2760938960),
 ('James Cameron', 2548546718),
 ('Francis Lawrence', 2476979588),
 ('Pierre Coffin', 2452006832),
 ('Steven Spielberg', 2449700791),
 ('Joss Whedon', 2424593677),
 ('Gore Verbinski', 2271362290),
 ('Sam Raimi', 2254066354),
 ('Lee Unkrich', 2081614145),
 ('Ridley Scott', 2044035909),
 ('Andrew Adamson', 1957706346),
 ('Chris Columbus', 1906968952),
 ('Carlos Saldanha', 1882719105),
 ('Tim Burton', 1801663457),
 ('Eric Darnell', 1755054393),
 ('Sam Mendes', 1710352791),
 ('Tom McGrath', 1705388064),
 ('Pete Docter', 1682867609),
 ('Chris Renaud', 1648116186),
 ('James Wan', 1620441367),
 ('Brad Bird', 1581913958),
 ('Conrad Vernon', 1578269902),
 ('Shawn Levy', 1515787063),
 ('Justin Lin', 1472947074),
 ('Steven Soderbergh', 1450928588),
 ('Ron Howard', 1385437233),
 ('Bill Condon', 1376664544),
 ('Colin Trevorrow', 1363528810),
 

In [99]:
answers['13'] = '5. Peter Jackson'#+

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [46]:
data[['director_set', 'genre_set']].apply(
        lambda x: Counter({name: ('Action' in x.genre_set) for name in x.director_set }), axis='columns'
    ).sum().most_common()

[('Robert Rodriguez', 9),
 ('Michael Bay', 7),
 ('Paul W.S. Anderson', 7),
 ('Antoine Fuqua', 6),
 ('Ridley Scott', 6),
 ('Brett Ratner', 5),
 ('Quentin Tarantino', 5),
 ('Zack Snyder', 5),
 ('Roland Emmerich', 5),
 ('Andrzej Bartkowiak', 5),
 ('Louis Leterrier', 5),
 ('Tony Scott', 5),
 ('Paul Greengrass', 5),
 ('Peter Jackson', 5),
 ('Lee Tamahori', 5),
 ('Gore Verbinski', 5),
 ('Rob Cohen', 5),
 ('Peter Berg', 5),
 ('J.J. Abrams', 4),
 ('Lilly Wachowski', 4),
 ('Lana Wachowski', 4),
 ('Olivier Megaton', 4),
 ('Guy Ritchie', 4),
 ('Bryan Singer', 4),
 ('Tim Story', 4),
 ('Renny Harlin', 4),
 ('Mark Neveldine', 4),
 ('Brian Taylor', 4),
 ('Dominic Sena', 4),
 ('Justin Lin', 4),
 ('Christopher Nolan', 4),
 ('Simon West', 4),
 ('John Moore', 4),
 ('Steven Spielberg', 4),
 ('Martin Campbell', 4),
 ('John Singleton', 4),
 ('Len Wiseman', 4),
 ('Brad Bird', 3),
 ('Matthew Vaughn', 3),
 ('Neill Blomkamp', 3),
 ('Jaume Collet-Serra', 3),
 ('Breck Eisner', 3),
 ('Pierre Morel', 3),
 ('David A

In [100]:
answers['14'] = '3. Robert Rodriguez'#+

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [48]:
data[data.release_year == 2012][['cast_set', 'profit']].apply(
        lambda x: Counter({name: x.profit for name in x.cast_set }), axis='columns'
    ).sum().most_common()

[('Chris Hemsworth', 1542450773),
 ('Denis Leary', 1319460639),
 ('Mark Ruffalo', 1299557910),
 ('Chris Evans', 1299557910),
 ('Scarlett Johansson', 1299557910),
 ('Robert Downey Jr.', 1299557910),
 ('Anne Hathaway', 1211851057),
 ('Sacha Baron Cohen', 1097110577),
 ('Ralph Fiennes', 1059561013),
 ('Seann William Scott', 967234366),
 ('Kristen Stewart', 919384318),
 ('Daniel Craig', 908561013),
 ('Javier Bardem', 908561013),
 ('Naomie Harris', 908561013),
 ('Judi Dench', 908561013),
 ('Josh Hutcherson', 872710235),
 ('Tom Hardy', 858717867),
 ('Gary Oldman', 831041287),
 ('Michael Caine', 831041287),
 ('Christian Bale', 831041287),
 ('Jennifer Lawrence', 800949406),
 ('Ray Romano', 782244782),
 ('Chris Wedge', 782244782),
 ('John Leguizamo', 779138503),
 ('Martin Freeman', 767003568),
 ('Andy Serkis', 767003568),
 ('Richard Armitage', 767003568),
 ('Ian McKellen', 767003568),
 ('Cate Blanchett', 767003568),
 ('Emma Thompson', 753009983),
 ('Elizabeth Reaser', 709000000),
 ('Peter Facin

In [101]:
answers['15'] = '3. Chris Hemsworth'#+

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [50]:
data[data.budget > data.budget.mean()].cast_set.apply(Counter).sum().most_common()

[('Matt Damon', 18),
 ('Adam Sandler', 17),
 ('Angelina Jolie', 16),
 ('Tom Cruise', 15),
 ('Samuel L. Jackson', 15),
 ('Eddie Murphy', 15),
 ('Mark Wahlberg', 14),
 ('Hugh Jackman', 14),
 ('Ben Stiller', 14),
 ('Jamie Foxx', 14),
 ('Russell Crowe', 14),
 ('Johnny Depp', 13),
 ('Jude Law', 13),
 ('Bruce Willis', 13),
 ('Will Smith', 13),
 ('Dwayne Johnson', 12),
 ('Ian McKellen', 12),
 ('Brad Pitt', 12),
 ('Owen Wilson', 12),
 ('Cameron Diaz', 12),
 ('Nicolas Cage', 12),
 ('Robert De Niro', 12),
 ('Leonardo DiCaprio', 11),
 ('Daniel Craig', 11),
 ('Robert Downey Jr.', 11),
 ('Steve Buscemi', 11),
 ('Vin Diesel', 10),
 ('Mark Ruffalo', 10),
 ('Jack Black', 10),
 ('Denzel Washington', 10),
 ('Emma Watson', 10),
 ('Gary Oldman', 10),
 ('Tom Hanks', 10),
 ('Orlando Bloom', 10),
 ('Ralph Fiennes', 9),
 ('Cate Blanchett', 9),
 ('Michael Caine', 9),
 ('Paul Bettany', 9),
 ('Ewan McGregor', 9),
 ('Brendan Gleeson', 9),
 ('Shia LaBeouf', 9),
 ('Nicole Kidman', 9),
 ('Colin Farrell', 9),
 ('Rach

In [102]:
answers['16'] = '3. Matt Damon'#+

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [52]:
data[data.cast_set.apply(lambda x: 'Nicolas Cage' in x)].genre_set.apply(Counter).sum().most_common()

[('Action', 17),
 ('Thriller', 15),
 ('Drama', 12),
 ('Crime', 10),
 ('Fantasy', 8),
 ('Adventure', 7),
 ('Comedy', 6),
 ('Science Fiction', 4),
 ('Mystery', 3),
 ('Family', 3),
 ('Animation', 3),
 ('History', 2),
 ('War', 1),
 ('Horror', 1),
 ('Romance', 1)]

In [103]:
answers['17'] = '2. Action'#+

# 18. Самый убыточный фильм от Paramount Pictures

In [56]:
data[data.production_companies.str.contains(r'Paramount\s*Pictures')].sort_values('profit')
    

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,2002-07-19,6.0,2002,7,-64831034,"{Drama, Thriller, History}",{Kathryn Bigelow},"{John Shrapnel, Peter Sarsgaard, Joss Ackland,...","{Palomar Pictures (II), National Geographic So..."
1123,tt0300556,80000000,19480739,Timeline,Paul Walker|Frances O'Connor|Gerard Butler|Bil...,Richard Donner,They had to travel into the past to save the f...,A group of archaeological students become trap...,116,Action|Adventure|Science Fiction,Paramount Pictures|Donners' Company|Mutual Fil...,2003-11-26,5.1,2003,11,-60519261,"{Adventure, Action, Science Fiction}",{Richard Donner},"{Paul Walker, Frances O'Connor, Gerard Butler,...","{Donners' Company, Cobalt Media Group, Artists..."
1722,tt0435705,70000000,18211013,Next,Nicolas Cage|Julianne Moore|Jessica Biel|Thoma...,Lee Tamahori,"If you can see the future, you can save it.",Las Vegas showroom magician Cris Johnson has a...,96,Action|Science Fiction|Thriller,Paramount Pictures|Virtual Studios|Revolution ...,2007-04-24,5.9,2007,4,-51788987,"{Thriller, Action, Science Fiction}",{Lee Tamahori},"{Thomas Kretschmann, Tory Kittles, Nicolas Cag...","{Revolution Studios, Saturn Films, Virtual Stu..."
1623,tt0375173,60000000,13395939,Alfie,Jude Law|Marisa Tomei|Omar Epps|Jane Krakowski...,Charles Shyer,Meet a man who never met a woman he didn't love.,"In Manhattan, the British limousine driver Alf...",103,Comedy,Paramount Pictures|Patalex Productions,2004-10-22,5.4,2004,10,-46604061,{Comedy},{Charles Shyer},"{Omar Epps, Jane Krakowski, RenÃ©e Taylor, Jud...","{Patalex Productions, Paramount Pictures}"
1631,tt0315297,50000000,25195000,Twisted,Ashley Judd|Samuel L. Jackson|Andy GarcÃ­a|Dav...,Philip Kaufman,Every murder has a mark.,Recently promoted and transferred to the homic...,97,Action|Adventure|Crime|Drama|Thriller,Paramount Pictures|Kopelson Entertainment|Harl...,2004-02-27,5.7,2004,2,-24805000,"{Drama, Crime, Action, Thriller, Adventure}",{Philip Kaufman},"{David Strathairn, Russell Wong, Andy GarcÃ­a,...","{Blackout Productions Inc., Harlequin Pictures..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,tt1229238,145000000,694713380,Mission: Impossible - Ghost Protocol,Tom Cruise|Jeremy Renner|Simon Pegg|Paula Patt...,Brad Bird,No Plan. No Backup. No Choice.,In the 4th installment of the Mission Impossib...,133,Action|Thriller|Adventure,Paramount Pictures|Skydance Productions|Bad Ro...,2011-12-07,6.7,2011,12,549713380,"{Adventure, Thriller, Action}",{Brad Bird},"{Simon Pegg, Tom Cruise, Jeremy Renner, Paula ...","{Bad Robot, Skydance Productions, TC Productio..."
1738,tt0418279,150000000,709709780,Transformers,Shia LaBeouf|Josh Duhamel|Megan Fox|Rachael Ta...,Michael Bay,Their war. Our world.,"Young teenager, Sam Witwicky becomes involved ...",144,Adventure|Science Fiction|Action,Paramount Pictures|DreamWorks SKG|Amblin Enter...,2007-06-27,6.6,2007,6,559709780,"{Adventure, Action, Science Fiction}",{Michael Bay},"{Rachael Taylor, Josh Duhamel, Tyrese Gibson, ...","{Di Bonaventura Pictures, Paramount Pictures, ..."
603,tt0367882,185000000,786636033,Indiana Jones and the Kingdom of the Crystal S...,Harrison Ford|Cate Blanchett|Shia LaBeouf|Ray ...,Steven Spielberg,The adventure continues . . .,"Set during the Cold War, the Soviets â€“ led b...",122,Adventure|Action,Lucasfilm|Paramount Pictures,2008-05-21,5.6,2008,5,601636033,"{Adventure, Action}",{Steven Spielberg},"{Ray Winstone, Harrison Ford, Cate Blanchett, ...","{Paramount Pictures, Lucasfilm}"
317,tt1055369,150000000,836297228,Trans

In [104]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'#+

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [58]:
data.groupby('release_year').profit.sum().sort_values(ascending=False)

release_year
2015    18668572378
2014    16397812953
2012    16077001687
2013    15243179791
2011    14730241341
2010    13531204961
2009    13423744372
2008    11663881990
2007    11565911801
2004     9634180720
2003     9228823312
2002     9002361487
2005     8981925558
2006     8691077320
2001     7950614865
2000     6101399805
Name: profit, dtype: int64

In [105]:
answers['19'] = '5. 2015'#+

# 20. Какой самый прибыльный год для студии Warner Bros?

In [60]:
data[
        data.production_companies.str.contains(r'Warner\s*Bros')
    ].groupby('release_year').profit.sum().sort_values(ascending=False)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

In [106]:
answers['20'] = '1. 2014'#+

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [63]:
Counter(data.release_month).most_common()

[(9, 227),
 (12, 190),
 (10, 186),
 (8, 161),
 (3, 156),
 (4, 149),
 (6, 147),
 (11, 146),
 (7, 142),
 (5, 140),
 (2, 135),
 (1, 110)]

In [107]:
answers['21'] = '4. Сентябрь'#+

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [65]:
len(data.query('6 <= release_month <= 8'))

450

In [108]:
answers['22'] = '2. 450'#+

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [68]:
data[
        data.release_month.isin({12, 1, 2})
    ].director_set.apply(Counter).sum().most_common()

[('Peter Jackson', 7),
 ('Clint Eastwood', 6),
 ('Steven Soderbergh', 6),
 ('Shawn Levy', 4),
 ('Nancy Meyers', 4),
 ('Martin Scorsese', 4),
 ('Adam Shankman', 4),
 ('Ridley Scott', 3),
 ('Rob Marshall', 3),
 ('Tim Burton', 3),
 ('Steve Carr', 3),
 ('Patrick Lussier', 3),
 ('Donald Petrie', 3),
 ('Marc Lawrence', 3),
 ('Lasse HallstrÃ¶m', 3),
 ('Edward Zwick', 3),
 ('Jason Friedberg', 3),
 ('Aaron Seltzer', 3),
 ('Peter Farrelly', 3),
 ('Steven Spielberg', 3),
 ('Wayne Wang', 3),
 ('Quentin Tarantino', 2),
 ('Adam McKay', 2),
 ('Michael Mann', 2),
 ('Glenn Ficarra', 2),
 ('John Requa', 2),
 ('Pierre Morel', 2),
 ('Wes Anderson', 2),
 ('Paul Thomas Anderson', 2),
 ('Jaume Collet-Serra', 2),
 ('George Clooney', 2),
 ('Shana Feste', 2),
 ('Ken Kwapis', 2),
 ('Henry Selick', 2),
 ('P.J. Hogan', 2),
 ('David S. Goyer', 2),
 ('Paul McGuigan', 2),
 ('Marcus Nispel', 2),
 ('Andrzej Bartkowiak', 2),
 ('Scott Cooper', 2),
 ('Allen Hughes', 2),
 ('Garry Marshall', 2),
 ('David O. Russell', 2),
 (

In [115]:
answers['23'] = '5. Peter Jackson'#+

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [71]:
data2 = data.copy()
data2['company_list'] = data2.company_set.apply(list)
data2 = data2.explode('company_list').rename(columns={'company_list': 'company'}).copy()

data2['title_length'] = data2.original_title.apply(len)

data2.groupby('company').title_length.max().sort_values(ascending=False).to_dict()

{'Four By Two Productions': 83,
 'Twentieth Century Fox Film Corporation': 83,
 'Walden Media': 62,
 'Walt Disney': 62,
 'Jim Henson Company, The': 59,
 '21 Laps Entertainment': 59,
 'Walt Disney Pictures': 59,
 'Warner Bros.': 58,
 'Plan B Entertainment': 58,
 'Virtual Studios': 58,
 'Alberta Film Entertainment': 58,
 'Scott Free Productions': 58,
 'Fox 2000 Pictures': 56,
 'Dune Entertainment': 56,
 'Jerry Bruckheimer Films': 54,
 '1492 Pictures': 50,
 'Lucasfilm': 50,
 'Paramount Pictures': 50,
 'Sunswept Entertainment': 50,
 'Ingenious Film Partners': 50,
 'WingNut Films': 49,
 'The Saul Zaentz Company': 49,
 'New Line Cinema': 49,
 'Miramax': 47,
 'Nickelodeon Movies': 47,
 'Universal Pictures': 47,
 'Dos Corazones': 47,
 'Kumar Mobiliengesellschaft mbH & Co. Projekt Nr. 2 KG': 47,
 'Samuel Goldwyn Films': 47,
 'Scott Rudin Productions': 47,
 'DreamWorks Pictures': 47,
 'Edward R. Pressman Film': 46,
 'Tim Burton Productions': 46,
 'Parkes+MacDonald Image Nation': 46,
 'Museum Can

In [109]:
answers['24'] = '5. Four By Two Productions'#+

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [82]:
data2 = data.copy()
data2['company_list'] = data2.company_set.apply(list)
data2 = data2.explode('company_list').rename(columns={'company_list': 'company'}).copy()

data2['word_count'] = data2.overview.str.replace(r'[,.!?/|\()\[\]~-]', r' ').apply(lambda x: len(str(x).split()))

data2.groupby('company').word_count.mean().sort_values(ascending=False).to_dict()                           

{'Midnight Picture Show': 175.0,
 '98 MPH Productions': 167.0,
 'Room 9 Entertainment': 166.0,
 'Heineken Branded Entertainment': 161.0,
 'Brookwell-McNamara Entertainment': 159.0,
 'Lions Gate Family Entertainment': 155.0,
 'Crest Animation Productions': 155.0,
 'Art Pictures Studio': 148.0,
 'Sanctum Australia': 145.0,
 'Great Wight Productions/ Osford Films': 145.0,
 'Wayfare Entertainment': 145.0,
 'Starbucks Entertainment': 144.0,
 'Cinema Gypsy Productions': 144.0,
 'Sudden Storm Productions': 136.0,
 'Magical Elves Productions': 134.0,
 'Scooter Braun Films': 134.0,
 'Disney Double Dare You (DDY)': 132.0,
 'K. JAM Media': 131.0,
 'Silent Hill DCP Inc.': 126.0,
 'Konami': 126.0,
 'Radical Studios': 125.5,
 'Omega Entertainment': 120.0,
 'Pantheon Entertainment Corporation': 120.0,
 'Innocence Productions': 120.0,
 'Oceana Media Finance': 120.0,
 'David Ladd Films': 120.0,
 'O.N.C. Entertainment': 118.0,
 'Ex3 Productions': 118.0,
 'Crying Box Productions': 117.0,
 'Ten Thirteen P

In [110]:
answers['25'] = '3. Midnight Picture Show'#+

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [85]:
data[data.vote_average > data.vote_average.quantile(.99)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month,profit,genre_set,director_set,cast_set,company_set
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,2015-06-09,8.0,2015,6,678708609,"{Family, Animation, Comedy}",{Pete Docter},"{Amy Poehler, Richard Kind, Lewis Black, Bill ...","{Pixar Animation Studios, Walt Disney Studios ..."
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,2015-10-16,8.0,2015,10,29401758,"{Drama, Thriller}",{Lenny Abrahamson},"{Joan Allen, Brie Larson, Jacob Tremblay, Sean...","{No Trace Camping, Duperele Films, Element Pic..."
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,Paramount Pictures|Legendary Pictures|Warner B...,2014-11-05,8.0,2014,11,456752480,"{Drama, Adventure, Science Fiction}",{Christopher Nolan},"{Michael Caine, Casey Affleck, Matthew McConau...","{Warner Bros., Legendary Pictures, Lynda Obst ..."
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,2014-07-30,7.9,2014,7,603312399,"{Adventure, Action, Science Fiction}",{James Gunn},"{Dave Bautista, Zoe Saldana, Vin Diesel, Bradl...","{Revolution Sun Studios, Marvel Studios, Bulle..."
125,tt2084970,14000000,233555708,The Imitation Game,Benedict Cumberbatch|Keira Knightley|Matthew G...,Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,Black Bear Pictures|Bristol Automotive,2014-11-14,8.0,2014,11,219555708,"{War, Drama, Thriller, History}",{Morten Tyldum},"{Benedict Cumberbatch, Rory Kinnear, Allen Lee...","{Bristol Automotive, Black Bear Pictures}"
128,tt2267998,61000000,369330363,Gone Girl,Ben Affleck|Rosamund Pike|Carrie Coon|Neil Pat...,David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,Twentieth Century Fox Film Corporation|Regency...,2014-10-01,7.9,2014,10,308330363,"{Drama, Mystery, Thriller}",{David Fincher},"{Ben Affleck, Neil Patrick Harris, Rosamund Pi...","{Pacific Standard, TSG Entertainment, Twentiet..."
138,tt2278388,30000000,174600318,The Grand Budapest Hotel,Ralph Fiennes|Tony Revolori|F. Murray Abraham|...,Wes Anderson,A perfect holiday without leaving home.,The Grand Budapest Hotel tells of a legendary ...,99,Comedy|Drama,Fox Searchlight Pictures|Scott Rudin Productio...,2014-02-26,7.9,2014,2,144600318,"{Drama, Comedy}",{Wes Anderson},"{Tony Revolori, F. Murray Abraham, Ralph Fienn...","{Fox Searchlight Pictures, TSG Entertainment, ..."
370,tt1375666,160000000,825500000,Inception,Leonardo DiCaprio|Joseph Gordon-Levitt|Ellen P...,Christopher Nolan,Your mind is the scene of the crime.,"Cobb, a skilled thief who commits corporate es...",148,Action|Thriller|Science Fiction|Mystery|Adventure,Legendary Pictures|Warner Bros.|Syncopy,2010-07-14,7.9,2010,7,665500000,"{Mystery, Action, Adventure, Thriller, Science...",{Christopher Nolan},"{Ellen Page, Leonardo DiCaprio, Tom Hardy, Ken...","{Warner Bros., Legendary Pictures, Syncopy}"
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Ca

In [111]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'#+

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [112]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint' #+ guessed answer no solution

# Submission

In [117]:
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '7': '5. The Lone Ranger (tt1210819)',
 '6': '5. Avatar (tt0499549)',
 '27': '5. Daniel Radcliffe & Rupert Grint',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '23': '5. Peter Jackson'}

In [116]:
len(answers)

27